In [1]:
import speech_recognition as sr
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
import time

Dataset sederhana

In [2]:
data_latih = [
    ("buka gorden", "BUKA"),
    ("buka tirai", "BUKA"),
    ("tolong buka gordennya", "BUKA"),
    ("biarkan cahaya masuk", "BUKA"),
    ("gelap banget buka dong", "BUKA"),
    ("tutup gorden", "TUTUP"),
    ("tutup tirai", "TUTUP"),
    ("tutup jendelanya", "TUTUP"),
    ("saya mau tidur tutup gorden", "TUTUP"),
    ("silau tutup dong", "TUTUP"),
    ("halo", "UNKNOWN"),
    ("apa kabar", "UNKNOWN")
]

kalimat, label = zip(*data_latih)

Membuat dan melatih model

In [4]:
model = make_pipeline(CountVectorizer(), MultinomialNB())
model.fit(kalimat, label)
print("Model NLP sudah siap")

Model NLP sudah siap


In [5]:
def dengarkan_perintah():
    recognizer = sr.Recognizer()
    
    with sr.Microphone() as source:
        print("\n🎤 Mendengarkan... (Silakan bicara: 'Buka gorden' atau 'Tutup gorden')")
        
        # Kalibrasi noise agar lebih akurat
        recognizer.adjust_for_ambient_noise(source, duration=1)
        
        try:
            # Menangkap audio
            audio = recognizer.listen(source, timeout=5)
            print("⏳ Sedang memproses suara...")
            
            # Mengubah Suara ke Teks (Menggunakan Google Speech API - butuh internet)
            # language='id-ID' penting agar dia paham bahasa Indonesia
            teks_hasil = recognizer.recognize_google(audio, language='id-ID')
            
            return teks_hasil.lower() # Ubah ke huruf kecil agar cocok dengan data latih
            
        except sr.WaitTimeoutError:
            print("⚠️ Waktu habis, tidak ada suara terdeteksi.")
            return None
        except sr.UnknownValueError:
            print("⚠️ Maaf, suara tidak jelas.")
            return None
        except sr.RequestError:
            print("⚠️ Masalah koneksi internet (Google API butuh internet).")
            return None

In [6]:
def jalankan_sistem_suara():
    print("-" * 40)
    print("SISTEM KONTROL GORDEN BERBASIS SUARA")
    print("Katakan 'keluar' atau 'berhenti' untuk mematikan program.")
    print("-" * 40)

    while True:
        # 1. Dapatkan teks dari suara
        perintah_teks = dengarkan_perintah()
        
        if perintah_teks:
            print(f"🗣️ Anda berkata: '{perintah_teks}'")
            
            # Cek untuk keluar program
            if "keluar" in perintah_teks or "berhenti" in perintah_teks:
                print("👋 Sampai jumpa!")
                break

            # 2. Prediksi Intent menggunakan Model NLP
            prediksi = model.predict([perintah_teks])[0]
            probabilitas = np.max(model.predict_proba([perintah_teks]))
            
            # 3. Logika Eksekusi
            # Kita set ambang batas keyakinan (misal 50%)
            if probabilitas > 0.5 and prediksi != "UNKNOWN":
                if prediksi == "BUKA":
                    print(">>> ✅ AKSI: MOTOR BERGERAK -> MEMBUKA GORDEN")
                elif prediksi == "TUTUP":
                    print(">>> ✅ AKSI: MOTOR BERGERAK -> MENUTUP GORDEN")
            else:
                print(">>> 🤖 Bot: Maaf, saya tidak mengerti perintah itu.")
        
        # Jeda sebentar sebelum mendengarkan lagi
        time.sleep(1)

if __name__ == "__main__":
    jalankan_sistem_suara()

----------------------------------------
SISTEM KONTROL GORDEN BERBASIS SUARA
Katakan 'keluar' atau 'berhenti' untuk mematikan program.
----------------------------------------

🎤 Mendengarkan... (Silakan bicara: 'Buka gorden' atau 'Tutup gorden')
⏳ Sedang memproses suara...
🗣️ Anda berkata: 'buka gorden'
>>> ✅ AKSI: MOTOR BERGERAK -> MEMBUKA GORDEN

🎤 Mendengarkan... (Silakan bicara: 'Buka gorden' atau 'Tutup gorden')
⏳ Sedang memproses suara...
⚠️ Maaf, suara tidak jelas.

🎤 Mendengarkan... (Silakan bicara: 'Buka gorden' atau 'Tutup gorden')
⏳ Sedang memproses suara...
🗣️ Anda berkata: 'tutup gorden'
>>> ✅ AKSI: MOTOR BERGERAK -> MENUTUP GORDEN

🎤 Mendengarkan... (Silakan bicara: 'Buka gorden' atau 'Tutup gorden')
⏳ Sedang memproses suara...
🗣️ Anda berkata: 'buka kode'
>>> ✅ AKSI: MOTOR BERGERAK -> MEMBUKA GORDEN

🎤 Mendengarkan... (Silakan bicara: 'Buka gorden' atau 'Tutup gorden')
⏳ Sedang memproses suara...
⚠️ Maaf, suara tidak jelas.

🎤 Mendengarkan... (Silakan bicara: 'Buka gor

KeyboardInterrupt: 